# **Flower Classification Model**



### Import Dataset and Dependencies

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !unzip /content/gdrive/MyDrive/flowers-dataset.zip -d /content/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Prepare Dataset ###

In [ ]:
# Fetch flowers directory
flowers_dataset = pathlib.Path("/content/flowers")
flowers_dataset

In [ ]:
# Confirm number of images
no_of_images = len(list(flowers_dataset.glob('*/*.jpg')))
no_of_images

In [ ]:
# count no of categories
categories = train_set.class_names
len(categories)

In [ ]:
# Create train set
train_set = tf.keras.preprocessing.image_dataset_from_directory(
  flowers_dataset,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(180, 180),
  batch_size=32)

In [ ]:
# Create validation set
validation_set = tf.keras.preprocessing.image_dataset_from_directory(
  flowers_dataset,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(180, 180),
  batch_size=32)

In [ ]:
# Fetch flower categories
categories = train_set.class_names
print(categories)

In [ ]:
# Remove bottlenecks while training model
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

###Standardize the data###

In [ ]:
normalized_train_set = train_set.map(lambda x, y: (layers.experimental.preprocessing.Rescaling(1./255)(x), y))
images, labels = next(iter(normalized_train_set))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))


### Create Model

In [ ]:
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(180, 180, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(categories))
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

### Training model

In [ ]:
epochs=10
history = model.fit(
  train_set,
  validation_data=validation_set,
  epochs=epochs
)

Epoch 1/10
108/108 [==============================] - 76s 703ms/step - loss: 1.3461 - accuracy: 0.4270 - val_loss: 1.0470 - val_accuracy: 0.5689
Epoch 2/10
108/108 [==============================] - 75s 696ms/step - loss: 1.0138 - accuracy: 0.6083 - val_loss: 0.9865 - val_accuracy: 0.5956
Epoch 3/10
108/108 [==============================] - 75s 698ms/step - loss: 0.8447 - accuracy: 0.6795 - val_loss: 0.9130 - val_accuracy: 0.6304
Epoch 4/10
108/108 [==============================] - 76s 700ms/step - loss: 0.6635 - accuracy: 0.7548 - val_loss: 1.0535 - val_accuracy: 0.5991
Epoch 5/10
108/108 [==============================] - 76s 700ms/step - loss: 0.5038 - accuracy: 0.8214 - val_loss: 0.9653 - val_accuracy: 0.6454
Epoch 6/10
108/108 [==============================] - 76s 704ms/step - loss: 0.3052 - accuracy: 0.8990 - val_loss: 1.0849 - val_accuracy: 0.6582
Epoch 7/10
108/108 [==============================] - 76s 703ms/step - loss: 0.1805 - accuracy: 0.9424 - val_loss: 1.4644 - val_ac

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()